# Agenda

0. Q&A
1. `**kwargs` and other parameter types
2. Scoping (LEGB)
3. Enclosing functions
4. Dispatch tables
5. Comprehensions
6. Sorting and `lambda` (and passing functions as arguments to other functions)

In [1]:
d = {}
d['a'] = 10
hash('a') % 8

0

In [2]:
hash('b') % 8

2

In [3]:
hash('c') % 8

6

In [4]:
hash('d') % 8

3

In [5]:
for one_letter in 'abcdefghij':
    print(f'{one_letter}: {hash(one_letter) % 8}')

a: 0
b: 2
c: 6
d: 3
e: 0
f: 5
g: 6
h: 0
i: 6
j: 4


In [6]:
d['e'] = 500

In [7]:
d = {'a':10, 'b':20, 'c':30}

new_stuff = {'b':400, 'c':900, 'd':1600}

d.update(new_stuff)   # this modifies d, the "receiving" dict, to take all key-value pairs from new_stuff

d

{'a': 10, 'b': 400, 'c': 900, 'd': 1600}

In [8]:
d = {'a':10, 'b':20, 'c':30}
new_stuff = {'b':400, 'c':900, 'd':1600}

# we can, as of 3.10 (?), use the | operator 

d | new_stuff   # this returns a new dict based merging d + new_stuff together, but doesn't modify d or new_stuff

{'a': 10, 'b': 400, 'c': 900, 'd': 1600}

In [9]:
d

{'a': 10, 'b': 20, 'c': 30}

In [10]:
# if you do want to modify d, you can use |=

d |= new_stuff    # this is (I think) identical in behavior to dict.update

In [11]:
d

{'a': 10, 'b': 400, 'c': 900, 'd': 1600}

# Parameter types for functions

1. Mandatory parameters (positional or keyword arguments)
2. Optional parameters (positional or keyword arguments), with a default argument value stored in the function's `__defaults__` tuple
3. `*args`, where `args` is a tuple containing all of the positional arguments that no other parameter got